In [ ]:
import sys
from datetime import datetime, timedelta
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

sys.path.append('/glade/u/home/danieldas/scratch/phd_stuff/daSilvaInvariants')
import numpy as np
import pylab as plt
import seaborn as sns
from astropy import units
from dasilva_invariants import diagnostics, invariants, meshes


In [ ]:
sns.set_style('darkgrid')

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
DPI = 500

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
#mesh = meshes.get_lfm_hdf4_data('../../data/LFMRCM-20131002_RBSP/ElkStorm-LR_mhd_2013-10-01T23-10-00Z.hdf')
mesh = meshes.get_lfm_hdf4_data('/glade/u/home/danieldas/scratch//data/LFM-20131002_RBSP/ElkStorm_mhd_2013-10-04T00-00-00Z.hdf')

In [ ]:
r_min, r_max = 5, 13
th = 0

plt.figure(figsize=(16, 10), dpi=DPI)
rs = np.arange(r_min, r_max + 1)
cmap = plt.get_cmap('viridis')

for i, r in enumerate(rs):
    x, y = np.cos(np.deg2rad(th)) * r, np.sin(np.deg2rad(th)) * r
    result = invariants.calculate_K(mesh, (x, y, 0), 7.5, step_size=None)

    plt.plot(np.rad2deg(result.trace_latitude), result.trace_field_strength, ',-',
             label=r'$\vec{r}=(' + ('%d'%r) + ', 0, 0)$', color='k')

plt.xlabel('Magnetic Latitude (degrees)')
plt.ylabel('|B| (Gauss)')
plt.title('Dayside Magnetic Field Line Trace from LFM', fontweight='bold')
plt.yscale('log')
plt.grid(color='#ccc', linestyle='dashed')
plt.legend()

In [ ]:
r_min, r_max = 10.5, 10.5
th = 0

plt.figure(figsize=(16, 10), dpi=DPI)
rs = np.arange(r_min, r_max + 1)
cmap = plt.get_cmap('viridis')

for i, r in enumerate(rs):
    x, y = np.cos(np.deg2rad(th)) * r, np.sin(np.deg2rad(th)) * r
    result = invariants.calculate_K(mesh, (x, y, 0), 7.5, step_size=None)

    plt.plot(np.rad2deg(result.trace_latitude), result.trace_field_strength, ',-',
             label=r'$\vec{r}=(' + ('%d' % r) + ', 0, 0)$', color='k')

plt.xlabel('Magnetic Latitude (degrees)')
plt.ylabel('|B| (Gauss)')
plt.title('Dayside Magnetic Field Line Trace from LFM (Quiet-time), Distance of 10.25 $R_E$', fontweight='bold')
plt.yscale('log')
plt.grid(color='#ccc', linestyle='dashed')

In [ ]:
def add_earth(ax):
    earth = plt.Circle((0, 0), 1, color='#4470AD')
    ax.add_patch(earth)


In [ ]:
import importlib
importlib.reload(diagnostics)

#kwargs = dict(xlim=(10, -10), ylim=(-7.5, 7.5), cbar_label=r'External Field Current Density $\rm{(nA/m^2)}$')
kwargs = dict(xlim=(13, -13), ylim=(-9, 9), cbar_label=r'External Field Current Density $\rm{(nA/m^2)}$')

In [ ]:
kwargs

In [ ]:
import glob
files = sorted(glob.glob('/glade/u/home/danieldas/scratch/data/LFM-20131002_RBSP/*mhd*.hdf'))
files[1000]

In [ ]:
Vsw = 591.5 
dist_L1_earth = 1467000 #https://www.johndcook.com/blog/2021/12/28/lagrange-points-l1-and-l2/
delay_sec = dist_L1_earth / Vsw
delay_sec/60

In [ ]:
#lfm_path = files[1000]
#time = datetime(2013, 10, 2, 7, 30) + timedelta(seconds=delay_sec)

lfm_path = '/glade/u/home/danieldas/scratch//data/LFM-20131002_RBSP/ElkStorm_mhd_2013-10-02T06-19-00Z.hdf'
time = datetime(2013, 10, 2, 6, 19)

In [ ]:
import importlib
from dasilva_invariants import utils
importlib.reload(meshes)
importlib.reload(diagnostics)
importlib.reload(utils)

In [ ]:
mesh_lfm = meshes.get_lfm_hdf4_data(lfm_path)
mesh_dip = meshes.get_dipole_mesh_on_lfm_grid(lfm_path)
mesh_sub = mesh_lfm.copy()
mesh_sub['B'] -= mesh_dip['B']
ax = diagnostics.meridional_plot_of_current(mesh_sub, 'LFM During Dst Minimum', **kwargs)
diagnostics.add_field_line_traces_meridional_plot(ax, mesh_lfm, color='white')
add_earth(ax)

In [ ]:
lfmrcm_path = lfm_path.replace('LFM', 'LFMRCM').replace('ElkStorm', 'ElkStorm-LR')
print(lfmrcm_path)
mesh_lfm = meshes.get_lfm_hdf4_data(lfmrcm_path)

mesh_dip = meshes.get_dipole_mesh_on_lfm_grid(lfmrcm_path)
mesh_sub = mesh_lfm.copy()
mesh_sub['B'] -= mesh_dip['B']

ax = diagnostics.meridional_plot_of_current(mesh_sub, 'LFM-RCM During Dst Minimum', **kwargs)
diagnostics.add_field_line_traces_meridional_plot(ax, mesh_lfm, color='white')
add_earth(ax)

In [ ]:
print('Getting T96')
mesh_t96 = meshes.get_tsyganenko_on_lfm_grid_with_auto_params(
    'T96', time, lfm_path, tell_params=True, external_field_only=True,
    force_zero_tilt=True,
    path='/glade/u/home/danieldas/scratch/data/WGhour-latest.d.zip'
)

In [ ]:
mesh_dip = meshes.get_dipole_mesh_on_lfm_grid(lfm_path)
mesh_total = mesh_dip.copy()
mesh_total['B'] += mesh_t96['B']
ax = diagnostics.meridional_plot_of_current(mesh_t96, 'T96 During Dst Minimum', **kwargs)
diagnostics.add_field_line_traces_meridional_plot(ax, mesh_total, color='white')
add_earth(ax)

In [ ]:
print('Getting T04')
mesh_t04 = meshes.get_tsyganenko_on_lfm_grid_with_auto_params(
    'T04', time, lfm_path, tell_params=True, external_field_only=True,
    force_zero_tilt=True,
    path='/glade/u/home/danieldas/scratch/data/WGhour-latest.d.zip'
)

In [ ]:
mesh_dip = meshes.get_dipole_mesh_on_lfm_grid(lfm_path)
mesh_total = mesh_dip.copy()
mesh_total['B'] += mesh_t04['B']
ax = diagnostics.meridional_plot_of_current(mesh_t96, 'TS05 During Dst Minimum', **kwargs)
diagnostics.add_field_line_traces_meridional_plot(ax, mesh_total, color='white')
add_earth(ax)